In [42]:
%run ./00-setup.ipynb

Cortex Python SDK v7.0.0


In [1]:
from cortex import Cortex, Message
import pandas as pd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import sys
import json
import re
import time
import datetime
import dill
import sklearn

client = Cortex.client()
exp = client.experiment('flowers/iris-predict')

exp

In [2]:
run = exp.get_run('yy4q0fg7')
model = run.get_artifact('model')

Using TensorFlow backend.


In [3]:
run = exp.last_run() 

model = run.get_artifact('model')

print('Run id: %s \nModel name: "%s"' %(run.id, model))

Run id: yy4q0fg7 
Model name: "<function adam_opt_model at 0x14a40a950>"


In [7]:
builder = client.builder()
builder.action('flowers/iris-predict')\
        .with_requirements(['tensorflow>=1.12,<2',
                            'keras>=2.2.4,<3'])\
        .from_model(model, model_type='tensorflow')\
        .build()

Building Cortex Action (function): flowers/iris-predict
Building Docker image private-registry.cortex.insights.ai/thatguyluke/flowers_iris-predict:qo93ejp...
Step 1/10 : FROM c12e/cortex-python36:7552534
Step 2/10 : WORKDIR /function
Step 3/10 : RUN conda config --add channels conda-forge
Step 4/10 : COPY conda_requirements.txt .
Step 5/10 : RUN conda install --yes --file conda_requirements.txt
Step 6/10 : RUN pip install "dill==0.2.8.2" "fdk==0.0.31" "cortex-client==7.0.0"
Step 7/10 : COPY requirements.txt .
Step 8/10 : RUN pip install -r requirements.txt
Step 9/10 : COPY action.py .
Step 10/10 : ENTRYPOINT ["python", "action.py"]
Successfully built 38dffbfb05d5
Successfully tagged private-registry.cortex.insights.ai/thatguyluke/flowers_iris-predict:qo93ejp
Image private-registry.cortex.insights.ai/thatguyluke/flowers_iris-predict:qo93ejp built in 0.297382 seconds
Pushing image to remote repository...
qo93ejp: digest: sha256:da4d6312e901e169c9e5ccf1fb9c4c438b65314a54e41eee0a1d45095d71

Name,Version,Type,Kind,Image,Deployment Status,flowers/iris-predict,6,function,python:3,private-registry.cortex.insights.ai/thatguyluke/flowers_iris-predict:qo93ejp,COMPLETED


In [8]:
action = client.action('flowers/iris-predict')
action

Name,Version,Type,Kind,Image,Deployment Status,flowers/iris-predict,6,function,python:3,private-registry.cortex.insights.ai/thatguyluke/flowers_iris-predict:qo93ejp,COMPLETED


In [17]:
params = {'columns': ['sepal_len','sepal_w','petal_len','petal_w'],
           'values': [4.9,3.1,1.5,0.2]}
frame = pd.DataFrame(params)
result = action.invoke(message=client.message(params))
print(result.payload)

2019-07-01 15:47:29,763 - INFO - cortex_client.client/client: Status: 500, Message: {"success":false,"error":"500 - {\"error\":{\"message\":\"Shape of passed values is (1, 4), indices imply (4, 4)\"}}"}
I0701 15:47:29.763242 4564891072 client.py:47] Status: 500, Message: {"success":false,"error":"500 - {\"error\":{\"message\":\"Shape of passed values is (1, 4), indices imply (4, 4)\"}}"}


HTTPError: 500 Server Error: Internal Server Error for url: https://api.cortex.insights.ai/v3/actions/flowers/iris-predict/invoke

In [12]:
x = np.argmax(result.payload['values'][0])

iris_dict = {0:'Iris-setosa',1:'Iris-versicolor',2:'Iris-virginica'}

iris_dict[x]

NameError: name 'result' is not defined

**Create Skill Using the Action**

To create a skill from the action, call the builder to get a skill builder.

In [ ]:
skill_builder = builder.skill('flowers/iris-predict')\
     .title('Iris Flower Prediction')\
     .description('Measurement based prediction for the type of Iris flowers.')

A resulting schema is used by the skill builder to identify elements of the input. In this example, the schema describes the measurement values for parts of the Iris flower.

In [ ]:
schema_name = 'flowers/iris-input-schema'

builder.schema(schema_name)\
     .title('Iris Prediction Schema Instance')\
     .from_parameters([{'name': 'values', 'type': 'array', 'format': 'float32'}]).build()

skill_builder = skill_builder.input('iris-measurement').title('Iris Measurements')\
     .use_schema(schema_name)\
     .all_routing(action, 'measurement-based-prediction').build()

Specify the skill construction. The construction describes the skill including a name that matches the action name and a description of the output.

In [ ]:
skill_builder = skill_builder.output('measurement-based-prediction').title('Classification')\
    .parameter(name='IrisClass', type='array', format='float32').build()

With the skill builder fully specified, call to_camel so the skill’s contents can be reviewed.

In [ ]:
skill_builder.to_camel()

**Publish the Skill**

Invoke the builder to publish the skill. Upon publication, the skill can be used in Cortex Studio with other skills to compose agents.

In [ ]:
skill = skill_builder.build()
print('%s (%s) v%d' % (skill.title, skill.name, skill.version))